Global setup



In [56]:
n=1
k=4
q=41 #Alkaline D
#q should be divisible by 8(?) for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2

In [57]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [58]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [59]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.765852644390328

Expected number of repetitions:

In [60]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

1.70494183535004

Alice generates a signing key

In [61]:
A = random_matrix(ZZ, 4, 4, x=0, y=q)
A

[21 20  3 10]
[28 39 23 30]
[30 11  7 39]
[10 31  8  7]

In [62]:
S1 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S1

[-1 -1 -1  1]
[ 1 -1  1  0]
[ 0  1  1 -1]
[ 1  1  1 -1]

In [63]:
S2 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S2

[ 1 -1 -1  0]
[ 1  0 -1  1]
[ 0  0  1  0]
[-1  1 -1 -1]

$(S_1, S_2)$ is the private key


In [64]:
T = matrix(((A*S1 + S2) % q))
T

[10 12 11  8]
[ 1 27 22 17]
[20  5 28 25]
[27 16 35 35]

$(A,T)$ is the public key


Frank tries to forge a message


In [65]:
M = 'FBOB'
M_list = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
M_list

[6, 2, 15, 2]

Pick random $\hat{z}_i$ and $\hat{c}$ and calculate $w$ (Is this necessary or could we go right for random $w$?)

In [66]:
z1hat = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
z1hat

[ 15]
[  6]
[-11]
[  1]

In [67]:
z2hat = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
z2hat

[ 5]
[ 1]
[-3]
[12]

In [68]:
clenbits = log(k,2)
clenbits

2

In [69]:
def sample_in_ball(digest):  # digest should be at least ****
    n = 2**clenbits  # variable name conflicts!  This n is local
    c = [0] * n
    k = n # This k is local
    for i in range(n-tau, n):
        while int(digest[clenbits*k:clenbits*(k+1)], 2) > i:
            k = k+1
        j = int(digest[clenbits*k:clenbits*(k+1)], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+tau-n], 2)
        k = k+1
    return c

In [70]:
chat = matrix(k,1,sample_in_ball(format(randrange(2^20),'020b'))) #20 random bits
chat

[ 0]
[-1]
[ 0]
[ 0]

In [71]:
w = matrix(A*z1hat+z2hat-T*chat).apply_map(lambda r: Mod(r,q).lift_centered())
w

[ 19]
[  8]
[-12]
[ -4]

Start the "non-cryptographic hash/XOF (combined?)"

In [72]:
w1list = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1list

[-5, 4, -20, -7]

In [73]:
def XOF(list1, list2, itemlen, numbits):
    out = ""
    list11 = list1
    while len(out)<numbits:
        newitem = vector(list11).dot_product(vector(list2)) % (2**itemlen) # make sure all bits of input are used
        out = out + format(newitem, f'0{itemlen}b')
        list11 = [newitem] + list11[:-1]
    return out[:numbits]

In [74]:
c_hash = XOF(M_list, w1list, 6, 20)
c_hash

'11000001011100011100'

Use XOF to sample a sparse polynomial

Generate the signature


In [75]:
c = matrix(k,1,sample_in_ball(c_hash))
c

[ 0]
[-1]
[ 0]
[ 0]

Now try to find $z_i$ that match $w$, i.e. $A\mathbf{z}_1+\mathbf{z}_2 = A\hat{z}_1 + \hat{z}_2 +ct - \hat{c}t$

Could guess $z_1$ at random and solve for $z_2$

In [77]:
z1 = random_matrix(ZZ, 4, 1, x=-(gamma1-beta-1), y=gamma1-beta)
z1

[  1]
[-13]
[ 11]
[ -9]

In [80]:
z2 = matrix(A*z1hat+z2hat+T*c-T*chat-A*z1).apply_map(lambda r: Mod(r,q).lift_centered())
z2

[ 16]
[-15]
[  1]
[ 20]

Is $\|\mathbf{z_1}\|_\infty \geq \gamma_1-\beta$?

In [23]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_1}$

In [24]:
z2 = matrix(y2 + S2*c); z2

[  1]
[-11]
[ 10]
[ -3]

Is $\|\mathbf{z_2}\|_\infty \geq \gamma_1-\beta$?

In [25]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_2}$

If not, $(\mathbf{z_1}, \mathbf{z_2}, \mathbf{c})$ is the signature


In [26]:
(z1, z2, c)

(
[-3]  [  1]  [ 0]
[-1]  [-11]  [-1]
[ 0]  [ 10]  [ 0]
[14], [ -3], [ 0]
)

Bob verifies the signature

Is $\|\mathbf{z_i}\|_\infty < \gamma_1-\beta$?

In [27]:
z1

[-3]
[-1]
[ 0]
[14]

In [28]:
z2

[  1]
[-11]
[ 10]
[ -3]

In [29]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [30]:
w1prime = matrix(((A*z1+z2-T*c) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1prime

[16]
[ 8]
[-6]
[ 2]

In [32]:
w1primelist = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1primelist

[16, 8, -6, 2]

In [33]:
cprime_hash = XOF(Mlist, w1primelist, 6, 20)
cprime_hash

'11000011011000001000'

In [34]:
cprime = matrix(k,1,sample_in_ball(cprime_hash))
cprime

[ 0]
[-1]
[ 0]
[ 0]

In [35]:
c == cprime

True

Yes.  The signature is accepted.